<a href="https://colab.research.google.com/github/amirmohammadkalateh/colony_counter/blob/main/colony_count_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def load_and_preprocess_data(image_paths, colony_counts, image_size=(128, 128)):
    """
    Loads images from given paths, preprocesses them, and prepares the data for training.

    Args:
        image_paths (list): List of paths to the image files.
        colony_counts (list): List of corresponding colony counts for each image.
        image_size (tuple): The target size (width, height) to resize the images.

    Returns:
        tuple: A tuple containing:
            - X (numpy.ndarray): Array of preprocessed image data.
            - y (numpy.ndarray): Array of colony counts.
    """
    X = []
    y = [] # Initialize y as an empty list
    for i, image_path in enumerate(image_paths): # Use enumerate to get the index
        try:
            # Load the image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Error: Could not read image at {image_path}")
                continue  # Skip to the next image
            # Resize the image
            img = cv2.resize(img, image_size)
            # Normalize pixel values to be between 0 and 1
            img = img / 255.0
            X.append(img)
            y.append(colony_counts[i]) # Append the corresponding count
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue  # Skip to the next image on error

    X = np.array(X)
    y = np.array(y)
    return X, y

def create_model(input_shape):
    """
    Creates a simple feedforward neural network model using Keras.

    Args:
        input_shape (tuple): Shape of the input data (e.g., (128, 128, 1) for grayscale images).

    Returns:
        keras.Model: The compiled neural network model.
    """
    model = keras.Sequential([
        # Flatten the 2D image data to a 1D vector
        keras.layers.Flatten(input_shape=input_shape),
        # Fully connected layer with 128 neurons and ReLU activation
        keras.layers.Dense(128, activation='relu'),
        # Fully connected layer with 64 neurons and ReLU activation
        keras.layers.Dense(64, activation='relu'),
        # Output layer with 1 neuron for predicting the colony count
        keras.layers.Dense(1, activation='linear')  # Use linear activation for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae']) # mae is Mean Absolute Error
    return model

def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
    """
    Trains the neural network model.

    Args:
        model (keras.Model): The neural network model to train.
        X_train (numpy.ndarray): Training data.
        y_train (numpy.ndarray): Training labels (colony counts).
        X_val (numpy.ndarray): Validation data.
        y_val (numpy.ndarray): Validation labels (colony counts).
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.

    Returns:
        History: The training history object.
    """
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1
    )
    return history

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained neural network model on the test data.

    Args:
        model (keras.Model): The trained neural network model.
        X_test (numpy.ndarray): Test data.
        y_test (numpy.ndarray): Test labels (colony counts).

    Returns:
        None
    """
    # Evaluate the model on the test data
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Mean Absolute Error: {mae:.4f}")

def predict_colony_count(model, image_path, image_size=(128, 128)):
    """
    Predicts the colony count for a single image.

    Args:
        model (keras.Model): The trained neural network model.
        image_path (str): Path to the image file.
        image_size (tuple): The target size to resize the image.

    Returns:
        int: The predicted colony count.
    """
    try:
        # Load and preprocess the image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image at {image_path}")
        img = cv2.resize(img, image_size)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        # Make a prediction
        prediction = model.predict(img)
        # Round the prediction to the nearest integer
        colony_count = int(round(prediction[0][0]))
        return colony_count
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return -1  # Return -1 to indicate an error

def main():
    """
    Main function to run the colony counting pipeline.
    """
    # 1. Prepare your data:
    #    - Collect a dataset of bacterial colony images and their corresponding colony counts.
    #    - Store the image paths and counts in lists.  For example:
    image_paths = [
        "path/to/image1.jpg",
        "path/to/image2.jpg",
        "path/to/image3.jpg",
        # Add more image paths...
    ]
    colony_counts = [25, 12, 30, ...]  # Corresponding colony counts

    # 2. Load and preprocess the data
    X, y = load_and_preprocess_data(image_paths, colony_counts)

    # 3. Split the data into training, validation, and test sets
    if len(X) > 0: # Check if X is not empty
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 of 0.8 = 0.2

        # Reshape the training data to add the channel dimension (for grayscale, it's 1)
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
        X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
        input_shape = X_train.shape[1:] # (128, 128, 1)

         # Scale the features
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
        X_val = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(X_val.shape)
        X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

        # 4. Create the model
        model = create_model(input_shape)

        # 5. Train the model
        history = train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32) # Increased epochs for better learning

        # 6. Evaluate the model
        evaluate_model(model, X_test, y_test)

        # 7. Make a prediction for a single image
        test_image_path = "path/to/your/test_image.jpg"  # Replace with a path to a test image
        predicted_count = predict_colony_count(model, test_image_path)
        if predicted_count != -1:
            print(f"Predicted colony count for {test_image_path}: {predicted_count}")
    else:
        print("No images were loaded. Please check the image paths and colony counts.")

if __name__ == "__main__":
    main()



Error: Could not read image at path/to/image1.jpg
Error: Could not read image at path/to/image2.jpg
Error: Could not read image at path/to/image3.jpg
No images were loaded. Please check the image paths and colony counts.


In [4]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def load_and_preprocess_data(image_paths, colony_counts, image_size=(128, 128)):
    """
    Loads images from given paths, preprocesses them, and prepares the data for training.

    Args:
        image_paths (list): List of paths to the image files.
        colony_counts (list): List of corresponding colony counts for each image.
        image_size (tuple): The target size (width, height) to resize the images.

    Returns:
        tuple: A tuple containing:
            - X (numpy.ndarray): Array of preprocessed image data.
            - y (numpy.ndarray): Array of colony counts.
    """
    X = []
    y = [] # Initialize y as an empty list
    for i, image_path in enumerate(image_paths): # Use enumerate to get the index
        try:
            # Load the image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Error: Could not read image at {image_path}")
                continue  # Skip to the next image
            # Resize the image
            img = cv2.resize(img, image_size)
            # Normalize pixel values to be between 0 and 1
            img = img / 255.0
            X.append(img)
            y.append(colony_counts[i]) # Append the corresponding count
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue  # Skip to the next image on error

    X = np.array(X)
    y = np.array(y)
    return X, y

def create_model(input_shape):
    """
    Creates a simple feedforward neural network model using Keras.

    Args:
        input_shape (tuple): Shape of the input data (e.g., (128, 128, 1) for grayscale images).

    Returns:
        keras.Model: The compiled neural network model.
    """
    model = keras.Sequential([
        # Flatten the 2D image data to a 1D vector
        keras.layers.Flatten(input_shape=input_shape),
        # Fully connected layer with 128 neurons and ReLU activation
        keras.layers.Dense(128, activation='relu'),
        # Fully connected layer with 64 neurons and ReLU activation
        keras.layers.Dense(64, activation='relu'),
        # Output layer with 1 neuron for predicting the colony count
        keras.layers.Dense(1, activation='linear')  # Use linear activation for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae']) # mae is Mean Absolute Error
    return model

def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
    """
    Trains the neural network model.

    Args:
        model (keras.Model): The neural network model to train.
        X_train (numpy.ndarray): Training data.
        y_train (numpy.ndarray): Training labels (colony counts).
        X_val (numpy.ndarray): Validation data.
        y_val (numpy.ndarray): Validation labels (colony counts).
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.

    Returns:
        History: The training history object.
    """
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1
    )
    return history

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained neural network model on the test data.

    Args:
        model (keras.Model): The trained neural network model.
        X_test (numpy.ndarray): Test data.
        y_test (numpy.ndarray): Test labels (colony counts).

    Returns:
        None
    """
    # Evaluate the model on the test data
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Mean Absolute Error: {mae:.4f}")

def predict_colony_count(model, image_path, image_size=(128, 128)):
    """
    Predicts the colony count for a single image.

    Args:
        model (keras.Model): The trained neural network model.
        image_path (str): Path to the image file.
        image_size (tuple): The target size to resize the image.

    Returns:
        int: The predicted colony count.
    """
    try:
        # Load and preprocess the image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image at {image_path}")
        img = cv2.resize(img, image_size)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        # Make a prediction
        prediction = model.predict(img)
        # Round the prediction to the nearest integer
        colony_count = int(round(prediction[0][0]))
        return colony_count
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return -1  # Return -1 to indicate an error

def main():
    """
    Main function to run the colony counting pipeline.
    """
    # 1. Prepare your data:
    #    - Collect a dataset of bacterial colony images and their corresponding colony counts.
    #    - Store the image paths and counts in lists.  For example:
    image_paths = [
        "path/to/image1.jpg",
        "path/to/image2.jpg",
        "path/to/image3.jpg",
        # Add more image paths...
    ]
    colony_counts = [25, 12, 30, ...]  # Corresponding colony counts

    # 2. Load and preprocess the data
    X, y = load_and_preprocess_data(image_paths, colony_counts)

    # 3. Split the data into training, validation, and test sets
    if len(X) > 0: # Check if X is not empty
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 of 0.8 = 0.2

        # Reshape the training data to add the channel dimension (for grayscale, it's 1)
        if len(X_train.shape) == 3:
            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
            X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
        elif len(X_train.shape) == 4:
            X_train = X_train
            X_val = X_val
            X_test = X_test
        else:
            print("Unexpected shape for input data.  Exiting.")
            return
        input_shape = X_train.shape[1:] # (128, 128, 1)

         # Scale the features
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
        X_val = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(X_val.shape)
        X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

        # 4. Create the model
        model = create_model(input_shape)

        # 5. Train the model
        history = train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32) # Increased epochs for better learning

        # 6. Evaluate the model
        evaluate_model(model, X_test, y_test)

        # 7. Make a prediction for a single image
        test_image_path = "path/to/your/test_image.jpg"  # Replace with a path to a test image
        predicted_count = predict_colony_count(model, test_image_path)
        if predicted_count != -1:
            print(f"Predicted colony count for {test_image_path}: {predicted_count}")
    else:
        print("No images were loaded. Please check the image paths and colony counts.")

if __name__ == "__main__":
    main()


Error: Could not read image at path/to/image1.jpg
Error: Could not read image at path/to/image2.jpg
Error: Could not read image at path/to/image3.jpg
No images were loaded. Please check the image paths and colony counts.


In [5]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

def load_and_preprocess_data(image_paths, colony_counts, image_size=(128, 128)):
    """
    Loads images from given paths, preprocesses them, and prepares the data for training.

    Args:
        image_paths (list): List of paths to the image files.
        colony_counts (list): List of corresponding colony counts for each image.
        image_size (tuple): The target size (width, height) to resize the images.

    Returns:
        tuple: A tuple containing:
            - X (numpy.ndarray): Array of preprocessed image data.
            - y (numpy.ndarray): Array of colony counts.
    """
    X = []
    y = [] # Initialize y as an empty list
    for i, image_path in enumerate(image_paths): # Use enumerate to get the index
        try:
            # Load the image in grayscale
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Error: Could not read image at {image_path}")
                continue  # Skip to the next image
            # Resize the image
            img = cv2.resize(img, image_size)
            # Normalize pixel values to be between 0 and 1
            img = img / 255.0
            X.append(img)
            y.append(colony_counts[i]) # Append the corresponding count
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue  # Skip to the next image on error

    X = np.array(X)
    y = np.array(y)
    return X, y

def create_model(input_shape):
    """
    Creates a simple feedforward neural network model using Keras.

    Args:
        input_shape (tuple): Shape of the input data (e.g., (128, 128, 1) for grayscale images).

    Returns:
        keras.Model: The compiled neural network model.
    """
    model = keras.Sequential([
        # Flatten the 2D image data to a 1D vector
        keras.layers.Flatten(input_shape=input_shape),
        # Fully connected layer with 128 neurons and ReLU activation
        keras.layers.Dense(128, activation='relu'),
        # Fully connected layer with 64 neurons and ReLU activation
        keras.layers.Dense(64, activation='relu'),
        # Output layer with 1 neuron for predicting the colony count
        keras.layers.Dense(1, activation='linear')  # Use linear activation for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae']) # mae is Mean Absolute Error
    return model

def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32):
    """
    Trains the neural network model.

    Args:
        model (keras.Model): The neural network model to train.
        X_train (numpy.ndarray): Training data.
        y_train (numpy.ndarray): Training labels (colony counts).
        X_val (numpy.ndarray): Validation data.
        y_val (numpy.ndarray): Validation labels (colony counts).
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for training.

    Returns:
        History: The training history object.
    """
    # Train the model
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        verbose=1
    )
    return history

def evaluate_model(model, X_test, y_test):
    """
    Evaluates the trained neural network model on the test data.

    Args:
        model (keras.Model): The trained neural network model.
        X_test (numpy.ndarray): Test data.
        y_test (numpy.ndarray): Test labels (colony counts).

    Returns:
        None
    """
    # Evaluate the model on the test data
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Mean Absolute Error: {mae:.4f}")

def predict_colony_count(model, image_path, image_size=(128, 128)):
    """
    Predicts the colony count for a single image.

    Args:
        model (keras.Model): The trained neural network model.
        image_path (str): Path to the image file.
        image_size (tuple): The target size to resize the image.

    Returns:
        int: The predicted colony count.
    """
    try:
        # Load and preprocess the image
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Could not read image at {image_path}")
        img = cv2.resize(img, image_size)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        # Make a prediction
        prediction = model.predict(img)
        # Round the prediction to the nearest integer
        colony_count = int(round(prediction[0][0]))
        return colony_count
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return -1  # Return -1 to indicate an error

def main():
    """
    Main function to run the colony counting pipeline.
    """
    # 1. Prepare your data:
    #    - Collect a dataset of bacterial colony images and their corresponding colony counts.
    #    - Store the image paths and counts in lists.  For example:
    image_paths = [
        "path/to/image1.jpg",  # Replace with your actual image paths
        "path/to/image2.jpg",
        "path/to/image3.jpg",
        # Add more image paths...
    ]
    colony_counts = [25, 12, 30, ...]  # Corresponding colony counts

    # 2. Load and preprocess the data
    X, y = load_and_preprocess_data(image_paths, colony_counts)

    # 3. Split the data into training, validation, and test sets
    if len(X) > 0: # Check if X is not empty
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 of 0.8 = 0.2

        # Reshape the training data to add the channel dimension (for grayscale, it's 1)
        if len(X_train.shape) == 3:
            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
            X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], X_val.shape[2], 1)
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
        elif len(X_train.shape) == 4:
            X_train = X_train
            X_val = X_val
            X_test = X_test
        else:
            print("Unexpected shape for input data.  Exiting.")
            return
        input_shape = X_train.shape[1:] # (128, 128, 1)

         # Scale the features
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
        X_val = scaler.transform(X_val.reshape(X_val.shape[0], -1)).reshape(X_val.shape)
        X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

        # 4. Create the model
        model = create_model(input_shape)

        # 5. Train the model
        history = train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32) # Increased epochs for better learning

        # 6. Evaluate the model
        evaluate_model(model, X_test, y_test)

        # 7. Make a prediction for a single image
        test_image_path = "path/to/your/test_image.jpg"  # Replace with a path to a test image
        predicted_count = predict_colony_count(model, test_image_path)
        if predicted_count != -1:
            print(f"Predicted colony count for {test_image_path}: {predicted_count}")
    else:
        print("No images were loaded. Please check the image paths and colony counts.")
        print("Make sure the paths in the 'image_paths' list are correct and that the images exist at those locations.") # Added debug message
        print("Also, ensure that the 'colony_counts' list has the same number of elements as the 'image_paths' list.") # Added debug message

if __name__ == "__main__":
    main()


Error: Could not read image at path/to/image1.jpg
Error: Could not read image at path/to/image2.jpg
Error: Could not read image at path/to/image3.jpg
No images were loaded. Please check the image paths and colony counts.
Make sure the paths in the 'image_paths' list are correct and that the images exist at those locations.
Also, ensure that the 'colony_counts' list has the same number of elements as the 'image_paths' list.
